In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
variacoes = pd.read_csv("../data/variacaocusto-metro-ingles.csv")

In [3]:
variacoes.head()

,Cód.,Unidade da Federação,january 2000,february 2000,march 2000,april 2000,may 2000,june 2000,july 2000,august 2000,...,july 2019,august 2019,september 2019,october 2019,november 2019,december 2019,january 2020,february 2020,march 2020,april 2020
0,11.0,Rondônia,1.34,1.32,1.52,1.74,1.75,1.85,2.06,2.27,...,4.77,5.42,5.58,5.72,5.69,5.80,0.40,0.55,1.12,1.29
1,12.0,Acre,0.62,0.63,1.68,1.33,0.95,1.25,1.63,2.23,...,2.56,3.07,3.33,3.44,3.77,4.37,0.36,0.62,0.74,0.71
2,13.0,Amazonas,-1.49,-0.89,-0.75,-0.54,-0.69,-0.71,-0.26,3.34,...,1.43,1.67,5.27,5.53,5.30,5.61,0.05,-0.02,0.17,0.20
3,14.0,Roraima,1.06,2.28,2.83,2.58,2.97,3.94,4.24,4.48,...,2.51,2.68,2.57,3.89,3.96,4.26,0.44,0.34,0.17,0.07
4,15.0,Pará,2.74,4.00,5.23,5.29,5.67,6.42,6.91,7.01,...,2.09,2.10,2.03,4.02,3.70,3.77,0.74,0.76,0.86,0.55


In [4]:
variacoes_long = variacoes.melt(id_vars=["Cód.", "Unidade da Federação"], 
        var_name="Date", 
        value_name="Value")

In [5]:
variacoes_long['datetime'] = pd.to_datetime(variacoes_long['Date'])

In [6]:
brutos = pd.read_csv("../data/custosmetro-2000-2020.csv").melt(id_vars=["Cód.", "Unidade da Federação"], 
        var_name="Date", 
        value_name="Value")

In [7]:
meses = {'janeiro': 'january','fevereiro': 'february','março': 'march','abril': 'april',
        'maio': 'may', 'junho': 'june', 'julho': 'july', 'agosto': 'august',
        'setembro': 'september', 'outubro': 'october', 'novembro': 'november', 'dezembro': 'december'}

In [8]:
#https://stackoverflow.com/questions/48824890/replace-words-in-pandas-dataframe-using-dictionary
d2 = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in meses.items()}
brutos['Date']=brutos['Date'].replace(d2, regex=True)

In [9]:
brutos_value = brutos[['Date','Value']]
full = pd.merge(brutos_value, variacoes_long, on='Date')
full.rename(columns={"Value_x": "bruto", "Value_y": "variacao"}, inplace=True)

In [10]:
#https://gist.github.com/ricardobarantini/5665214
estados = {
'Acre':'AC',
'Alagoas':'AL',
'Amapá':'AP',
'Amazonas':'AM',
'Bahia':'BA',
'Ceará':'CE',
'Distrito Federal':'DF',
'Espírito Santo':'ES',
'Goiás':'GO',
'Maranhão':'MA',
'Mato Grosso':'MT',
'Mato Grosso do Sul':'MS',
'Minas Gerais':'MG',
'Pará':'PA',
'Paraíba':'PB',
'Paraná':'PR',
'Pernambuco':'PE',
'Piauí':'PI',
'Rio de Janeiro':'RJ',
'Rio Grande do Norte':'RN',
'Rio Grande do Sul':'RS',
'Rondônia':'RO',
'Roraima':'RR',
'Santa Catarina':'SC',
'São Paulo':'SP',
'Sergipe':'SE',
'Tocantins':'TO'
}

In [11]:
d3 = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in estados.items()}
full['UF']=full['Unidade da Federação'].replace(d3, regex=True).replace('MT do Sul','MS')

In [12]:
full = full[['Cód.','UF', 'Unidade da Federação', 'bruto', 'variacao', 'datetime']]
full

,Cód.,UF,Unidade da Federação,bruto,variacao,datetime
0,11.0,RO,Rondônia,277.33,1.34,2000-01-01
1,12.0,AC,Acre,277.33,0.62,2000-01-01
2,13.0,AM,Amazonas,277.33,-1.49,2000-01-01
3,14.0,RR,Roraima,277.33,1.06,2000-01-01
4,15.0,PA,Pará,277.33,2.74,2000-01-01
...,...,...,...,...,...,...
177871,43.0,RS,Rio Grande do Sul,1227.27,1.18,2020-04-01
177872,50.0,MS,Mato Grosso do Sul,1227.27,0.61,2020-04-01
177873,51.0,MT,Mato Grosso,1227.27,2.18,2020-04-01
177874,52.0,GO,Goiás,1227.27,0.58,2020-04-01


In [13]:
full.to_csv('../data/tidy_custos.csv')
full.to_json(path_or_buf='../data/tidy_custos.json', 
            force_ascii=False,orient='table',index=False)